In [1]:
import numpy as np
import pandas as pd
import io, os, json, pickle
import re, pyarrow, boto3

In [ ]:
object = s3.Object('flexr-data','current-data/ECR/run-1691165518376-part-block-0-r-00000-snappy.parquet')
object.download_fileobj(io.BytesIO())
df = pd.read_parquet(io.BytesIO())

In [ ]:
#paths = [path for path in s3.ls("s3://path/to/file/") if path.endswith(".parquet")]
s3 = boto3.resource('s3')
bucket = s3.Bucket('flexr-data')
prefix_objs = bucket.objects.filter(Prefix='current-data/ECR/')
df = []
for obj in prefix_objs:
    if obj.key.endswith('.parquet'):
        body = obj.get()['Body'].read()
        temp = pd.read_parquet(io.BytesIO(body))
        df.append(temp)
df_total = pd.concat(df)
df_total.reset_index(drop=True, inplace=True)

In [66]:
# local 
ENWL1 = pd.read_csv('ENWL1.csv')
ENWL2 = pd.read_csv('ENWL2.csv')
NGED1 = pd.read_csv('NGED1.csv', usecols=range(0, 57)) 
NGED2 = pd.read_csv('NGED2.csv', usecols=range(0, 57))
NPG = pd.read_csv('NPG.csv') 
SPEN = pd.read_csv('SPEN.csv')
SSEN = pd.read_csv('SSEN.csv') 
UKPN1 = pd.read_csv('UKPN1.csv') 
UKPN2 = pd.read_csv('UKPN2.csv') 

C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3948624301.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  NGED1 = pd.read_csv('NGED1.csv', usecols=range(0, 57))
C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3948624301.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  NGED2 = pd.read_csv('NGED2.csv', usecols=range(0, 57))


In [69]:
NGED1.iloc[0,0]

1170000124890

In [ ]:
NPG.isnull().sum()

In [ ]:
pip install convertbng

In [17]:
from convertbng.util import convert_lonlat

In [70]:
NGED1.iloc[:, 10] = NGED1.iloc[:, 10].str.replace(',', '')
NGED1.iloc[:, 11] = NGED1.iloc[:, 11].str.replace(',', '')
NGED2.iloc[:, 10] = NGED2.iloc[:, 10].str.replace(',', '')
NGED2.iloc[:, 11] = NGED2.iloc[:, 11].str.replace(',', '')
ecr = [ENWL1, ENWL2, NGED1, NGED2, NPG, SPEN, SSEN, UKPN1, UKPN2]
ecr2=[]
for dfo in ecr: 
    print(dfo.shape)
    dfo.iloc[:,0] = pd.to_numeric(dfo.iloc[:,0], downcast='integer', errors='coerce')
    dfo.iloc[:,1] = pd.to_numeric(dfo.iloc[:,1], downcast='integer', errors='coerce')
    dfo.iloc[:,0] = dfo.iloc[:,0].astype('Int64')
    dfo.iloc[:,1] = dfo.iloc[:,1].astype('Int64')
    dfo.replace('DATA NOT AVAILABLE', np.nan, inplace=True)
    dfo.replace('Data not available', np.nan, inplace=True)
    dfo.replace('Data Not Available', np.nan, inplace=True)
    dfo.replace('data not available', np.nan, inplace=True)
    dfo.replace('data not Available', np.nan, inplace=True)
    dfo.replace('--redacted--', np.nan, inplace=True)
    dfo.replace('--Redacted--', np.nan, inplace=True)
    dfo.replace('--REDACTED--', np.nan, inplace=True)
    dfo.replace('redacted', np.nan, inplace=True)
    dfo.replace('Redacted', np.nan, inplace=True)
    dfo.replace('REDACTED', np.nan, inplace=True)
    dfo.replace('TBC', np.nan, inplace=True)
    dfo_notna=dfo.loc[dfo.iloc[:, 10].notna() & dfo.iloc[:, 11].notna(), :]
    df_isna=dfo.loc[dfo.iloc[:, 10].isna() | dfo.iloc[:, 11].isna(), :]
    lonlat = convert_lonlat(dfo_notna.iloc[:, 10].astype('float'), dfo_notna.iloc[:, 11].astype('float')) 
    dfo_notna['Longitude'], dfo_notna['Latitude'] = lonlat[0], lonlat[1]
    dfo_notna['Longitude'], dfo_notna['Latitude'] = dfo_notna['Longitude'].astype('string'), dfo_notna['Latitude'].astype('string')
    df_isna['Longitude'], df_isna['Latitude'] = np.nan, np.nan
    df_concat = pd.concat([dfo_notna, df_isna], axis=0)
    df_concat.replace('', np.nan, inplace=True)
    df_concat=df_concat.astype('string')
    print(df_concat.shape)
    ecr2.append(df_concat)

(1197, 58)
(1197, 60)
(517, 58)
(517, 60)
(7998, 57)


C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3449564913.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfo_notna['Longitude'], dfo_notna['Latitude'] = lonlat[0], lonlat[1]
C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3449564913.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfo_notna['Longitude'], dfo_notna['Latitude'] = lonlat[0], lonlat[1]
C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3449564913.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

(7998, 59)
(2998, 57)
(2998, 59)
(790, 58)
(790, 60)
(884, 58)
(884, 60)
(1416, 57)
(1416, 59)
(3896, 61)


C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3449564913.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfo_notna['Longitude'], dfo_notna['Latitude'] = lonlat[0], lonlat[1]
C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3449564913.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfo_notna['Longitude'], dfo_notna['Latitude'] = lonlat[0], lonlat[1]
C:\Users\Hang.Ruan\AppData\Local\Temp\ipykernel_24660\3449564913.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

(3896, 61)
(1085, 62)
(1085, 62)


In [71]:
[ENWL1_, ENWL2_, NGED1_, NGED2_, NPG_, SPEN_, SSEN_, UKPN1_, UKPN2_] = ecr2

In [72]:
ENWL1_.to_csv('ENWL1_.csv', index=False, encoding='utf-8')
ENWL2_.to_csv('ENWL2_.csv', index=False, encoding='utf-8')
NGED1_.to_csv('NGED1_.csv', index=False, encoding='utf-8')
NGED2_.to_csv('NGED2_.csv', index=False, encoding='utf-8')
NPG_.to_csv('NPG_.csv', index=False, encoding='utf-8')
SPEN_.to_csv('SPEN_.csv', index=False, encoding='utf-8')
SSEN_.to_csv('SSEN_.csv', index=False, encoding='utf-8')
UKPN1_.to_csv('UKPN1_.csv', index=False, encoding='utf-8')
UKPN2_.to_csv('UKPN2_.csv', index=False, encoding='utf-8')